# PA3

In this assignment, you will perform logistic regression on the MNIST dataset to detect which number is written in an image! Each image in the MNIST data shows a handwritten number, and your goal is to use logistic regression to detect when a number is an "8" or a "1".

Each input example ($x_i$ from class) is a vector referring to the brightness of each pixel in an image. Your logistic regression will learn a weight vector whose length is equal to the number of pixels in each image.

First, you will train logistic regression to recognize the digit 8. You'll do this once with coordinate descent (CD) and once with SGD (discussed in class). Then, you'll train logistic regression to recognize the digit 1, using your preferred method between CD and SGD.

Fill in the ACT's below. Writing asserts to test your code is recommended, but not graded for this assignment.

In [1]:
# Install Tensorflow before calling the following line (for anaconda, run "conda install tensorflow")
# (for a native Python installation, you can try pip or pip3 install tensorflow)

# Import the MNIST data
from tensorflow.keras.datasets import mnist as keras_mnist
(X_train, y_train), (X_test, y_test) = keras_mnist.load_data()

/Users/arpj/anaconda3/envs/cos324/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arpj/anaconda3/envs/cos324/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arpj/anaconda3/envs/cos324/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/arpj/anaconda3/envs/cos324/lib/python3.7/s

11493376/11490434 [==============================] - 132s 12us/step


In [2]:
# There are 60000 training examples, each example is a 28 by 28 pixel
#   grayscale image, represented by a 28 by 28 array of numbers
print('Shape of X_train: ', X_train.shape)

Shape of X_train:  (60000, 28, 28)


## Log-Loss and Its Gradient

In lecture, we used the convention that negative examples are labeled $-1$ and positive examples are labeled $+1$. That is, $y_i\in\{-1,+1\}$. We found that, if $\mathcal{L}(\mathbf{w})$ is our loss on $\mathbf{w}$, and $|S|$ is the number of examples,
$$\mathcal{L}(\mathbf{w}) = \frac{1}{|S|} \sum_{i\in S} \log\big(1+e^{-z_i}\big)
~~ \mbox{ where } ~~
z_i = y_i (\mathbf{w} \cdot \mathbf{x}_i) ~~ .$$
This occurs when we let $\mathcal{L}(\mathbf{w})$ be the negative log probability of us predicting the right label. Note: Sometimes, in lecture we used $\mathcal{L}(\mathbf{w}) = \sum_{i\in S} \log(1+e^{-z_i})$, omitting the division by $|S|$. Since we generally assume that the number of examples $|S|$ is fixed, these two forms of the loss differ by a constant factor, so they are pretty much equivalent.

Alternatively, suppose we use a different convention, that negative examples are labeled $0$ instead. That is, $y_i\in\{0,1\}$. Just as in the previous case, we predict $+1$ with probability $\hat{y}_i = 1/(1+e^{-\mathbf{w} \cdot \mathbf{x_i}})$. Thus, we predict $0$ with probability $1 - \hat{y}_i = 1 - 1/(1+e^{-\mathbf{w} \cdot \mathbf{x_i}}) = 1/(1+e^{\mathbf{w} \cdot \mathbf{x_i}})$.

Then, observe that the probability of predicting the right label is equal to $\hat{y}_i$ when $y_i = 1$, and is equal to $1 - \hat{y}_i$ when $y_i = 0$. Then, do you see why the loss can be expressed as the following? (You can plug in $y_i = 1$ or $y_i = 0$.)
$$\mathcal{L}(\mathbf{w}) =
  -\frac{1}{|S|} \sum_{i\in S} \big(y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i)\big)
~~ \mbox{ where } ~~ 
\hat{y}_i = \frac{1}{1+e^{-\mathbf{w} \cdot \mathbf{x_i}}}
~~.$$

If we find the gradient for the first form (when $y_i\in\{-1,+1\}$), we obtain the following:
$$ \nabla_{\mathbf{w}} \mathcal{L} = -\frac{1}{|S|} \sum_{i\in S} q_i y_i \mathbf{x}_i 
~~ \mbox{ where } ~~ q_i = \frac{1}{1 + e^{z_i}} ~~ .
$$

For the second form (when $y_i\in\{0,+1\}$), we find that the gradient is:
$$ \nabla_{\mathbf{w}} \mathcal{L} = \frac{1}{|S|} \sum_{i\in S} \big(\hat{y}_i - y_i\big) \mathbf{x}_i  ~~ ,$$
where $\hat{y}_i$ is as defined above. You can verify these gradient values for yourself.

For your own practice (ungraded), you should confirm the equivalence of these two forms, and derive them for yourself. That is, calculate both gradients (see if you get the same thing we did). Then, verify that when you plug in $y_i = 1$ you get the same thing, and when you plug in $y_i = -1$ or $=0$ respectively, you get the same thing. If you're having trouble with this, please ask on Piazza!

Finally, implement the following functions for the case where $y_i\in\{-1,+1\}$. You will use these functions throughout this notebook.

In [25]:
import numpy as np

# utility function for z
def compute_z(X, y, w):
    return y * (np.dot(X, w))


# Calculate loss from margins
# Here, z is equal to y times X dot w
# That is, z_i = y_i * x_i dot w
def logloss_from_z(z):
    ### ACT 1
    return np.mean(np.log(1 + np.power(np.e, -z)))


# Calculate error from margins
# the error is the proportion of examples that you "get wrong".
# a.k.a. the examples where you assign less than 50% chance to the correct label
# This is equivalent to the proportion of examples where z_i is less than 0
def error_from_z(z):
    ### ACT 2
    wrong = z[z < 0]
    return wrong.size / z.size


# Calculate loss from parameter vector
def logloss(X, y, w):
    ### ACT 3
    z = compute_z(X, y, w)
    return logloss_from_z(z)


# Calculate error from parameter vector
def error(X, y, w):
    ### ACT 4
    z = compute_z(X, y, w)
    return error_from_z(z)


# Gradient of LogLoss w.r.t. w
def logloss_gradient(X, y, w):
    ### ACT 5
    # mean over which axis? is the result of a gradient a matrix, a vector, a scalar?
    z = compute_z(X, y, w)
    q = 1 / (1 + np.power(np.e, z))
    return -1 * np.mean(q * y * X)


# Calculate error & loss from parameter vector
# Return two floats in the format (loss, error)
def logloss_and_error(X, y, w):
    ### ACT 6
    return logloss(X, y, w), error(X, y, w)

## Bias

Normally, we initialize $w$ to be zero (as we discussed in the last assignment). This means we initially predict everything as 0.5 (a.k.a. 50-50). This might not make sense. For example, in the MNIST problem, we are distinguishing a single digit from the 9 other digits. On average, there's only a 10% chance each image is our desired digit, so initially maybe we should predict something lower.

This is where bias comes in. Bias is something that is added to every prediction. For example, if we have bias $b$, an example $x_i$, and a weight vector $\mathbf{w}$, we would predict the following probability:
$$\Pr[y_i = +1] = \frac{1}{1 + e^{-(\mathbf{w}\cdot \mathbf{x}_i + b)}}$$

So, in MNIST, we'd want to initialize bias to be negative, since initially we would predict that an average digit should have less than 50% chance to be classified as $+1$.

Bias is actually easy to add into our models.

Rather than creating a new variable $b$, we simply append $1$ to each example $\mathbf{x}_i$. (That is, we append a column of ones to the data matrix $X$.) Since the length of $\mathbf{x}_i$ has increased by one, the length of our weight vector $\mathbf{w}$ has also increased by one. Let's call the final entry of our weight vector $w_d$, and call the final entry of our example $x_{i, d}$ (so $x_{i, d} = 1$). Then, when we compute $\mathbf{w}\cdot \mathbf{x}_i$, we get
$$\mathbf{w} \cdot \mathbf{x}_i = w_1x_{i, 1} + w_2x_{i, 2} + \ldots + w_dx_{i, d}$$
which equals the previous dot product, plus $w_dx_{i, d}$ which equals $w_d$.

Thus, $w_{d}$ becomes the bias term: it is added to every dot product regardless of the example $\mathbf{x}_i$.

## Setting Initial Bias Using Log-Odds (ACT's 7-9)
As mentioned above, we don't necessarily want to initialize the bias term of $w$, $w_{d}$, to zero. Below is a small math exercise where you will find the best value to initialize bias.

Total number of positive examples is $n_+$ (pos) and negative $n_-$ (neg). Assume $\mathbf{w}=\mathbf{0}$ except for the weight of the last feature $w_{d}$. Since we have the other entries of $w$ initialized equal to $0$, we can write the logistic loss as the following:
$$\frac1n \sum_{i:y_i=+1} \log(1+\exp(-w_d)) + \frac1n \sum_{i:y_i=-1} \log(1+\exp(w_d))$$
If you're unclear on how we got this, verify it by plugging in $\mathbf{w}$ into the formula for logistic loss. Most of the terms in the dot product equal zero except for the last term in the dot product. For reference: $\exp(-w_d) = e^{-w_d}$.

To find the best initialization for $w_d$, we take the derivative of the above loss, and set it to $0$. The ACT's below ask you to do this, to find the best initial bias. Submit your answer as a LaTeX'd answer within this notebook cell (or make a new notebook cell).

ACT 7: Take the derivative, with respect to $w_{d}$, of the above expression for logistic loss.

ACT 8: Set the derivative of this to zero, and solve for w_d.

ACT 9: Fill in the value of $w_d$ below which minimizes your initial loss, then code it below. (We've given you a hint, the answer is a log of something.)
$$
w_d = \log\left( \mathbf{\mbox{ACT 9}} \right) ~.
$$

In [ ]:
# Calculate the initial bias
def init_bias(pos, neg):
   ### ACT 9 continued

## Image normalization and flattening

Before working with MNIST, we need to normalize and flatten the input images. After all, we do not normally perform logistic regression on 2D input data like images, we normally perform logistic regression on 1D data! So, we first flatten the image so it becomes 1-dimensional.

First we flatten the data set of images: each $p_x \times p_y$ image becomes a 1d vector of size $d = p_x \, p_y$.

This amounts to reshaping $X$ from a $n \times p_x \times p_y$ tensor to a $n \times d$ matrix.

In [ ]:
def flatten_images(X):
    ### ACT 10

Next, we want to normalize each image. This is useful because some images may be overall darker or lighter, or higher- or lower-contrast than others. We don't want this to affect our classification. Thus, we normalize each image by making sure each image's average "brightness" (a.k.a. pixel value) is 0, and each image's pixels have standard deviation one.

Finally, we want to provide the option to append a bias column of ones to the data matrix.

In your method below, normalize each image, by subtracting the average value of that image over all its pixels, and
dividing by the standard deviation of the image's pixel values. If the bias argument is non-zero, add a bias term by appending `bias` to each example. For example, if `bias=1`, then append an entry of $1$ to each example. Algebraicly, flattening and normalizing amounts to flattening each image to a $d$ dimensional vector
$\mathbf{x}$ and calculating average pixel value and variance of pixel values,
$$ m = \mathbb{E}(\mathbf{x}) = \frac 1 d \sum_{i=1}^d x_i ~~~
   s^2 = \mathbb{V}(\mathbf{x}) = \frac 1 d \sum_{i=1}^d x_i^2 - m^2 ~~ . $$
Here, $m$ is the image's mean and $s$ is the image's standard deviation. (You can use pre-existing numpy functions to compute these.)

Define $a=s^{-1}$, then the normalized image (represented as a vector) is, ${a (\mathbf{x} - m)}$.

In [ ]:
# Normalize each example as described above. If bias is non-zero, add a bias term by appending `bias` to
# each example
def normalize(X, bias=0):
    ### ACT 11

In [ ]:
# Normalize and flatten the training and test images
# You may print Xtr and Xte, or view their shape, to get a feeling for their structure
Xtr = normalize(flatten_images(X_train), bias=1)
Xte = normalize(flatten_images(X_test), bias=1)

Convert the train labels $\mathbf{y}^{\mbox{tr}}$ and test labels $\mathbf{y}^{\mbox{te}}$ to vectors over
$\{-1,+1\}$ indicating whether each example is an 8.

For example, `ytr8[i] = -1` if the i-th example is NOT an 8

Similarly to above, viewing the first few entries of ytr8,
may give you some intuition about their contents and structure

In [ ]:
ytr8 = (2 * (y_train == 8) - 1).reshape(len(y_train), 1)
yte8 = (2 * (y_test == 8) - 1).reshape(len(y_test), 1)

ACT 12: For coordinate descent use the mean value theorem to form a quadratic upper-bound on the loss for a single coordinate $j$. When using the convention of $y_i\in\{0,1\}$ show that,
    $$
     |S|\cdot\mathcal{L}(\mathbf{w}+\delta\mathbf{1}_j) \leq
       \kappa + \sum_i (\hat{y}_i - y_i) X_{ij} \, \delta + \frac18 \sum_i  X_{ij}^2 \, \delta^2 ~~.
    $$
Here we use $\mathbf{w}+\delta\mathbf{1}_j$ to denote $\mathbf{w}$ plus $\delta$ in the $j$-th coordinate. Hint: The slides titled "Mean Value Theorem" and "Putting It All Together" in the Logistic Regression lecture should be helpful. Please ask on Piazza if you're having trouble understanding ACT 12 or the relevant lecture slides!

Note: The factor of $|S|$ multiplied on the left side is used to make this equation similar to the one proved in lecture. In this notebook we divided error and loss by $|S|$ to get average error or loss. However, in the logistic regression lecture we tried to minimize total loss (a.k.a. we didn't divide loss by $|S|$). Minimizing these two losses _is equivalent_ since $|S|$ is just a constant factor, so we multiply by $|S|$ on the left to keep things consistent with lecture in this ACT.

ACT 13: Denote the $j^{\mbox{th}}$ column of $X$ by $\mathbf{v}_j$ and let $c_j= \|\mathbf{v}_j\|^2$.

Show that the $\delta^\star$ minimizing the above bound $\kappa + \sum_i (\hat{y}_i - y_i) X_{ij} \, \delta + \frac18 \sum_i  X_{ij}^2 \, \delta^2$ is,
$$
\delta^\star = \frac{4}{c_j} (\mathbf{y - \hat{y}) \cdot v_j} = \frac{4}{c_j} \sum_{i=1}^n (y_i - \hat{y}_i)\mathbf{v}_j[i] ~~.
$$

(Note that $\mathbf{v}_j[i] = X_{ij}$.) Hint: Take the derivative and set it equal to zero.

ACT 14: Of course the above expression assumes that $y_i\in \{0, 1\}$, which is different from our current setting, where $y_i \in \{-1, 1\}$.

To convert back to the $\{-1, 1\}$ setting, consider the positive and negative examples in the above summation. Positive examples have $y_i = 1$, so that term in the summation becomes $(1 - \hat{y}_i)\mathbf{v}_j[i]$. Negative examples have $y_i = 0$, so that term in the summation becomes $(-\hat{y}_i)\mathbf{v}_j[i]$.

Can you rewrite the expression for $\delta^\star$ when $y_i\in\{-1, 1\}$?

Hint: First, re-write $(y_i - \hat{y}_i)$ from the previous setting in terms of the new $y_i$ and $z_i$, or $y_i$ and $q_i$. You should get a simple expression that replaces $(y_i - \hat{y}_i)$. The other terms in the expression for $\delta^\star$ shouldn't change, since they don't depend on $y_i$. Thus, you can just plug in your new expression for $(y_i - \hat{y}_i)$ into the previous expression for $\delta^\star$.

In [ ]:
# `x` is the j-th column of X (a.k.a. the v_j from above)
# `y` is the vector of labels
# `z` is a vector with entries z_i
# `cj` is c_j from above, the squared norm of `x`
def delta_wj(x, y, z, cj):
    ### ACT 15 (code your result from ACT 14 here)

In [ ]:
from numpy.random import randint
from numpy.random import permutation

# You don't need to use this class yourself: we provided the index-sampling code for you
# sample new index (with or without replacement)
# d is the max index
class IndexSampler:
    def __init__(self, d):
        self.d = d
        self.prm = None
    
    def sample_new_index(self, replace = 1):
        if replace:
            return randint(self.d)
        if self.prm is None:
            self.prm = permutation(self.d)
            self.head = 0
        ind = self.prm[self.head]
        self.head += 1
        if self.head == self.d:
            self.head = 0
            self.prm = None
        return ind

In [ ]:
# epochs is maximum number of epochs to train
# eps is your termination condition number, similar to in PA2's linear regression
# Every epoch, report the loss (we've provided some code that reports loss for you)
# (An epoch consists of d updates)

### ACT 16
def logistic_regression_cd(X, y, epochs=100, eps=0.001):
    pstr = 'Epoch: {0:2d}  Loss: {1:5.3f}  Error: {2:5.3f}'
    n, d = X.shape
    ### initialize w, z, c, errors, and losses
    ### c is a vector whose j-th entry is equal to c_j from above
    ### errors should be a vector containing the error at each step
    ### losses should be a vector containing the loss at each step
    ### NOTE: Don't forget to initialize the last entry of w as initial bias!
    cur_epoch = 0
    sampler = IndexSampler(d)
    for e in range(1, d * epochs + 1):
        ### we've chosen a coordinate for you, now perform coordinate descent below
        j = sampler.sample_new_index(replace)
        if e % d == 0:
            ### update losses and errors with the current loss and error
            cur_epoch += 1
            print(pstr.format(cur_epoch, losses[-1], errors[-1] * 100))
            if (losses[-2] - losses[-1]) / losses[-1] < eps: break
    print('\n')
    return w, losses, errors

In [ ]:
[w, loss, err] = logistic_regression_cd(Xtr, ytr8, epochs=20)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss of your logistic classifier over time
# The y-axis should be loss, and the x-axis should be the epoch
# The plot doesn't need to be pretty, just show the loss going down over time!
### ACT 17

In [ ]:
# plot the error of your logistic classifier over time 
### ACT 18

In [ ]:
pstr = 'Test  Loss: {0:5.3f}  Error: {1:5.3f}'

In [ ]:
# Evaluate the loss and error of your weight vector on the test data and test labels
# (Don't update your weight vector here: this is purely to evaluate it)
test_loss, test_err = ### ACT 19

In [ ]:
print(pstr.format(test_loss, test_err))

In [ ]:
# Here, we visualize w for you
# First, we bound the entries of w within 3 standard deviations
# (in order to ignore outliers which would mess up the image)
image_w = np.maximum(np.minimum(w, 3 * np.std(w)), -3 * np.std(w))
# Next, we re-shape image_w into the original 28 by 28 shape of the image
image_w = image_w[0:-1].reshape(28, 28)

In [ ]:
# Visualize image_w
plt.axis('off')
plt.imshow(image_w, cmap='gray')

In [ ]:
# `ind` is the indices in Xtest of images which are "8" but classified incorrectly
#   by your classifier
ind = (np.argwhere(((yte8 == 1) * (np.matmul(Xte, w) < 0))))[:,0]

In [ ]:
# Display 25 images of "8"s which your classifier incorrectly classified as non-8
# This kind of visualization can be useful if you're wondering what sorts of images give
#   your classifier trouble
ncols, nrows = 5, 5
fig, axes = plt.subplots(ncols, nrows, figsize=(1.5*ncols, 1.5*nrows))
for i in range(ncols * nrows):
    ax = axes[i//ncols, i%ncols]
    x = X_test[ind[i],:,:].reshape(28,28)
    plt.axis('off')
    plt.tight_layout()
    ax.imshow(x, cmap='gray')

In [ ]:
# Now that you've completed coordinate descent, you will implement SGD!

# A handle is a convenient way to pass in many arguments/specifications
# to your function. In this case, the logistic SGD handle holds
# parameters such as the gradient function, the loss function, and learning rate.

### ACT 20
# Fill in the missing parameters in the handle below, and experiment with
# different values for them!
# For the final submission, use the parameters you found worked best, except set eps = 0.001
# (We will not be too picky with your parameters but we do expect them
# to be good enough for your SGD to properly learn)
# Instead of eta==0 use a real learning rate, (hint: try 0.01, 0.05, and 0.1)
# Experiment also with a decreasing learning rate: eta[t] = eta / sqrt(c + t) for eta=1 and c=10
def prepare_logistic_sgd_handle():
    h = dict()
    h['pstr'] = 'Epoch: {0:2d}  Loss: {1:5.3f}  Error: {2:5.3f}'
    h['epochs'] = 100
    # rather than a single value, eta is an array here, containing the eta for each epoch
    h['eta'] = 0.00 * np.ones(h['epochs'])
    h['grad'] = logloss_gradient
    h['loss'] = logloss
    h['error'] = error
    ### Adjust this batch size if you wish. A size of 1000 should be fine, though.
    h['batch_size'] = 1000
    ### You can play with eps, but in your final submission set it to 0.001, so
    ### you can achieve the desired accuracy.
    h['eps'] = 0.001
    return h

In [ ]:
# Implement the termination condition (it can be similar to your
#   termination condition for coordinate descent)
# We recommend you don't terminate when c_loss is greater than p_loss, though
def terminate(p_loss, c_loss, eps):
    ### ACT 21

In [ ]:
### ACT 22
# Implement logistic regression with SGD
# h is the handle you defined above
def sgd(X, y, h):
    loss, error, grad, eta = h['loss'], h['error'], h['grad'], h['eta']
    epochs, bs = h['epochs'], h['batch_size']
    eps, pstr = h['eps'], h['pstr']
    n, d = X.shape
    nbs = int(n / bs)
    sampler = IndexSampler(nbs)
    ### ACT initialize w, losses, errors, and other variables you may need
    for e in range(1, epochs * nbs):
        # these two lines get a batch of examples and labels for you
        head = sampler.sample_new_index(replace=0) * bs
        Xt, yt = X[head:head + bs], y[head:head + bs]
        ### ACT find stochastic gradient using the functions above with (Xt, yt)
        ### ACT update w as appropriate
        if e % nbs == 0:
            ### ACT update losses and errors
            print(pstr.format(e // nbs, losses[-1], errors[-1]))
            if terminate(losses[-2], losses[-1], eps): break
    print('\n')
    return w, losses, errors

In [ ]:
[w_sgd, loss_sgd, error_sgd] = sgd(Xtr, ytr8, prepare_logistic_sgd_handle())

In [ ]:
# plot the loss of your SGD classifier over time
### ACT 23

In [ ]:
# plot the error of your SGD classifier over time
### ACT 24

In [ ]:
# plot the error of your SGD classifier and CD classifer on the same graph
### ACT 25

In [ ]:
# this is similar to above
ind = (np.argwhere(((yte8 == 1) * (np.matmul(Xte, w_sgd) < 0))))[:,0]

In [ ]:
# Display 25 images of "8"s which your classifier incorrectly classified as non-8
ncols, nrows = 5, 5
fig, axes = plt.subplots(ncols, nrows, figsize=(1.5*ncols, 1.5*nrows))
for i in range(ncols * nrows):
    ax = axes[i//ncols, i%ncols]
    x = X_test[ind[i],:,:].reshape(28,28)
    plt.axis('off')
    plt.tight_layout()
    ax.imshow(x, cmap='gray')

### Classifying the digit 1
Now, repeat most of the above steps, except using the digit 1 instead. In particular:
- Obtain training and test labels (yte1 and ytr1) (Xtr and Xte should remain the same)
- Train a new weight vector using either CD or SGD
- Visualize your weight vector for "1"
- Plot the error of your weight vector for 1 over time
- Display images of "1"s which your classifier incorrectly classified as non-1

In [ ]:
### ACT 26 (this may take multiple cells)